# Agentic RAG: Bauhaftpflicht-Fälle intelligent durchsuchen

## Überblick

In `4_rag` haben wir ein einfaches RAG-System gebaut:
- Frage rein → 1x Vektor-Suche → 5 Chunks → LLM antwortet

Das Problem: **Jede Frage wird gleich behandelt.** Egal ob du nach ähnlichen Fällen suchst, einen bestimmten Fall analysieren willst, oder eine Übersicht brauchst.

In `5_agentic_rag` geben wir einem **Agenten** mehrere **Tools** und lassen ihn **selbst entscheiden**, was er tut.

```
4_rag (fix):           Frage → Suche → Antwort
5_agentic_rag (flexibel): Frage → Agent denkt → Tool → Ergebnis → Agent denkt → evtl. nochmal Tool → Antwort
```

---

## 1. Das Problem: Alles in einer Datenbank

In `4_rag` war **alles in ChromaDB** gespeichert — sowohl die Dokumentinhalte als auch die Metadaten (Fall-ID, Cluster, Betrag, Status, etc.):

```
Bisherige Architektur (4_rag):

┌─────────────────────────────────────────────────────────┐
│                      ChromaDB                           │
│                                                         │
│  Chunk 1:                                               │
│    text:     "Wir erklären hiermit den Schadensfall..." │
│    vektor:   [0.23, 0.87, 0.12, ...]                   │
│    metadata: {case_id: W1, cluster: Wasser, CHF: 95k}  │
│                                                         │
│  Chunk 2:                                               │
│    text:     "Das Gericht entschied..."                 │
│    vektor:   [0.71, 0.33, 0.45, ...]                   │
│    metadata: {case_id: H2, cluster: HLKS, CHF: 2.8M}   │
│                                                         │
│  ... (112 Chunks)                                       │
└─────────────────────────────────────────────────────────┘
```

ChromaDB kann **semantisch suchen** ("Finde ähnliche Texte"), aber es kann **nicht**:

| Frage | Was nötig wäre | ChromaDB kann das? |
|-------|----------------|--------------------|
| "Wie viele Fälle gibt es?" | `COUNT(*)` | Nein |
| "Durchschnittlicher Betrag pro Cluster?" | `AVG() GROUP BY` | Nein |
| "Welche Fälle sind im Kanton BE?" | `WHERE kanton = 'BE'` | Nur eingeschränkt |
| "Teuerster Fall?" | `ORDER BY betrag DESC LIMIT 1` | Nein |
| "Wie viele Dokumente hat Fall W3?" | `COUNT(*) WHERE case_id = 'W3'` | Nein |

Das sind alles **strukturierte Fragen** — die klassische Domäne von SQL. Vektor-Datenbanken sind dafür nicht gemacht.

## 2. Die Lösung: Zwei Datenbanken, ein Agent

Wir trennen die Daten nach ihrer Natur:

```
Neue Architektur (5_agentic_rag):

┌─────────────────────────────────┐   ┌──────────────────────────────────┐
│          SQLite (cases.db)      │   │          ChromaDB                │
│                                 │   │                                  │
│  Strukturierte Metadaten:       │   │  Dokumentinhalte:                │
│                                 │   │                                  │
│  case_id │ cluster  │ betrag    │   │  chunk │ text          │ vektor  │
│  ────────┼──────────┼───────    │   │  ──────┼───────────────┼──────── │
│  W1      │ Wasser   │ 95'000    │   │  1     │ "Wir erklä..."│ [0.2…]  │
│  H2      │ HLKS     │ 2'836'548 │   │  2     │ "Das Geric..."│ [0.7…]  │
│  ...     │ ...      │ ...       │   │  ...   │ ...           │ ...     │
│                                 │   │                                  │
│  → COUNT, AVG, GROUP BY, WHERE  │   │  → "Finde ähnliche Texte"       │
│  → Zählen, Filtern, Sortieren   │   │  → Semantische Suche            │
└─────────────────┬───────────────┘   └───────────────┬──────────────────┘
                  │                                   │
                  └──────────┐     ┌──────────────────┘
                             │     │
                        ┌────┴─────┴────┐
                        │    Agent      │
                        │  (LLM)        │
                        │               │
                        │  Entscheidet: │
                        │  SQL oder     │
                        │  Vektor oder  │
                        │  beides?      │
                        └───────────────┘
```

### Warum die Trennung?

Jede Datenbank macht das, **was sie am besten kann**:

| | SQLite | ChromaDB |
|---|---|---|
| **Stärke** | Zählen, Filtern, Aggregieren | Bedeutung verstehen, Ähnlichkeit finden |
| **Daten** | Strukturiert (Tabellen, Zeilen, Spalten) | Unstrukturiert (Freitext, Dokumente) |
| **Fragetyp** | "Wie viele?", "Durchschnitt?", "Sortiert nach?" | "Gibt es ähnliche?", "Was steht drin?" |
| **Analogie** | Excel-Tabelle | Google-Suche |

Der **Agent** ist der "Sachbearbeiter", der weiss, wann er in welche Datenbank schauen muss — und der beides kombinieren kann.

## 3. Setup

### 3.1 SQLite-Datenbank erstellen

Zuerst indexieren wir die `case_bible.json`-Dateien in eine SQLite-Datenbank.
Die ChromaDB aus `4_rag` nutzen wir weiterhin — die ist schon fertig.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from indexer import index_all

count = index_all()
print(f"\n==> {count} Fälle in SQLite indexiert!")

### 3.2 Was steht jetzt in der SQLite-Datenbank?

Schauen wir rein. Wir haben zwei Tabellen:
- **cases**: Eine Zeile pro Fall (Metadaten)
- **documents**: Eine Zeile pro Dokument (welcher Typ, welches Datum)

In [ ]:
import sqlite3
import pandas as pd
from config import SQLITE_PATH

db = sqlite3.connect(str(SQLITE_PATH))

# Alle Fälle anzeigen
df_cases = pd.read_sql_query(
    "SELECT case_id, kanton, cluster, status, forderung_brutto, normen FROM cases",
    db
)
print("=== Tabelle 'cases' ===")
print(df_cases.to_string(index=False))

db.close()

In [ ]:
db = sqlite3.connect(str(SQLITE_PATH))

# Dokumente pro Fall
df_docs = pd.read_sql_query(
    """SELECT case_id, COUNT(*) as anzahl_docs, 
       GROUP_CONCAT(doc_typ, ', ') as dokument_typen
       FROM documents GROUP BY case_id LIMIT 5""",
    db
)
print("=== Tabelle 'documents' (erste 5 Fälle) ===")
print(df_docs.to_string(index=False))

db.close()

## 4. Was SQL jetzt kann, was ChromaDB nicht konnte

Hier einige Beispiele für strukturierte Abfragen, die vorher **unmöglich** waren:

In [ ]:
db = sqlite3.connect(str(SQLITE_PATH))

# Frage: "Wie viele Fälle gibt es pro Status?"
print("=== Fälle pro Status ===")
for row in db.execute("SELECT status, COUNT(*) as anzahl FROM cases GROUP BY status"):
    print(f"  {row[0]}: {row[1]}")

print()

# Frage: "Durchschnittliche Forderung pro Cluster?"
print("=== Durchschnittliche Forderung pro Cluster ===")
for row in db.execute("""
    SELECT cluster, 
           COUNT(*) as anzahl,
           CAST(AVG(forderung_brutto) AS INTEGER) as avg_forderung
    FROM cases 
    GROUP BY cluster 
    ORDER BY avg_forderung DESC
"""):
    print(f"  {row[0]}: {row[1]} Fälle, Ø CHF {row[2]:,}")

print()

# Frage: "Welcher Fall hat die höchste Forderung?"
print("=== Top 3 Fälle nach Forderung ===")
for row in db.execute("""
    SELECT case_id, cluster, forderung_brutto 
    FROM cases 
    ORDER BY forderung_brutto DESC 
    LIMIT 3
"""):
    print(f"  {row[0]}: {row[1]} — CHF {row[2]:,.0f}")

db.close()

## 5. Die drei Tools des Agenten

Unser Agent hat jetzt drei Tools — jedes greift auf eine andere Datenquelle zu:

```
┌──────────────────────┬────────────────────┬─────────────────────────────────┐
│ Tool                 │ Datenquelle        │ Wann nutzt der Agent es?        │
├──────────────────────┼────────────────────┼─────────────────────────────────┤
│ vector_search        │ ChromaDB           │ "Gibt es ähnliche Fälle mit..?" │
│                      │ (Embeddings)       │ "Was steht im Gutachten?"       │
├──────────────────────┼────────────────────┼─────────────────────────────────┤
│ get_case_overview    │ JSON-Dateien       │ "Erzähl mir alles über Fall W3" │
│                      │ (case_bible.json)  │ "Wer ist am Fall F1 beteiligt?" │
├──────────────────────┼────────────────────┼─────────────────────────────────┤
│ sql_query            │ SQLite (cases.db)  │ "Wie viele Fälle pro Cluster?"  │
│                      │                    │ "Durchschnittliche Forderung?"  │
│                      │                    │ "Welche Fälle im Kanton BE?"    │
└──────────────────────┴────────────────────┴─────────────────────────────────┘
```

### Das Entscheidende: Der Agent kombiniert die Tools

Bei einer komplexen Frage wie *"Welche Wasserschaden-Fälle gibt es und was steht in den Gutachten?"* kann der Agent:
1. **sql_query** → Findet alle Fälle im Cluster Wasser (strukturiert)
2. **vector_search** → Sucht in den Gutachten nach Details (semantisch)

Das konnte kein einzelnes Tool allein.

## 6. Den Agent erstellen und testen

In [ ]:
from agents import Agent, Runner
from tools import vector_search, get_case_overview, sql_query

INSTRUCTIONS = """Du bist ein Experte für Bauhaftpflicht-Fälle in der Schweiz.
Du hilfst Sachbearbeitern, ihre Fälle zu analysieren und vergleichbare Fälle zu finden.

Wichtige Regeln:
1. Nutze IMMER deine Tools, um Informationen zu finden — erfinde nichts.
2. Wenn du mehrere Fälle vergleichen sollst, hole jeden Fall einzeln.
3. Gib immer die Fall-ID(s) an, aus denen die Information stammt.
4. Antworte auf Deutsch.
5. Wenn du Beträge nennst, nutze das Format 'CHF 50'000'.
6. Wenn du nicht genug Information findest, sage das klar.

Deine Tools:
- vector_search: Für inhaltliche/semantische Suchen
- get_case_overview: Für Details zu einem bestimmten Fall
- sql_query: Für strukturierte Analysen (Zählungen, Summen, Durchschnitte)

Strategie:
- Für 'Wie viele?', 'Durchschnitt?', 'Welche Kantone?' → sql_query
- Für 'Gibt es ähnliche Fälle?', 'Was steht im Gutachten?' → vector_search
- Für 'Erzähl mir alles über Fall X' → get_case_overview
- Für komplexe Fragen: Kombiniere mehrere Tools
"""

agent = Agent(
    name="Bauhaftpflicht Agent",
    instructions=INSTRUCTIONS,
    tools=[vector_search, get_case_overview, sql_query],
)

### Test 1: Strukturierte Frage (→ sql_query)

Diese Frage verlangt Zählung und Gruppierung. Der Agent sollte `sql_query` nutzen.

In [ ]:
result = await Runner.run(agent, "Wie viele Fälle gibt es pro Cluster und was ist die durchschnittliche Forderung?")
print(result.final_output)

### Test 2: Inhaltliche Frage (→ vector_search)

Hier geht es um den Inhalt der Dokumente. Der Agent sollte `vector_search` nutzen.

In [ ]:
result = await Runner.run(agent, "Gibt es Fälle mit Wasserabdichtungsproblemen? Was steht in den Gutachten?")
print(result.final_output)

### Test 3: Spezifischer Fall (→ get_case_overview)

Hier will der Benutzer alles über einen bestimmten Fall wissen.

In [ ]:
result = await Runner.run(agent, "Was sind die wichtigsten Fakten zu Fall H2?")
print(result.final_output)

### Test 4: Kombinierte Frage (→ sql_query + vector_search)

Das ist die Stärke des Systems. Der Agent muss **beide Datenbanken** nutzen:

In [ ]:
result = await Runner.run(
    agent,
    "Welche Fälle sind noch im Prozess, wie hoch sind die Forderungen, und was sind die strittigen Punkte?"
)
print(result.final_output)

### Test 5: Vergleich zweier Fälle (→ get_case_overview + get_case_overview)

Der Agent muss zwei Fälle einzeln laden und vergleichen.

In [ ]:
result = await Runner.run(agent, "Vergleiche Fall W1 mit Fall H2. Was sind die Unterschiede bei Betrag, Status und Vorgehen?")
print(result.final_output)

## 7. Interaktiver Chat-Modus

Du kannst den Agent auch im interaktiven Modus nutzen.
Tippe deine Fragen ein und der Agent antwortet. Beende mit `exit`.

In [ ]:
from agents import run_demo_loop

await run_demo_loop(agent)

## 8. Zusammenfassung: Die Evolution des Systems

```
Projekt       Architektur                     Was es kann
─────────────────────────────────────────────────────────────────────
4_rag         ChromaDB only                   Ähnliche Texte finden
              Frage → Suche → Antwort         (immer gleicher Ablauf)

5_agentic_rag ChromaDB + SQLite + Agent       Alles von 4_rag PLUS:
              Frage → Agent entscheidet        - Zählen, Aggregieren
              → Tool(s) → Antwort              - Strukturierte Analysen
                                               - Tools kombinieren
                                               - Mehrstufige Abfragen
```

### Was wäre der nächste Schritt?

In einer produktiven Umgebung würde man:
- **SQLite** durch **BigQuery** oder **PostgreSQL** ersetzen (für Millionen von Fällen)
- **ChromaDB** durch **Qdrant** oder **Pinecone** ersetzen (für Millionen von Chunks)
- Aber die **Architektur bleibt identisch**: SQL-DB + Vektor-DB + Agent mit Tools

Das Schöne: Der Agent-Code ändert sich dabei kaum — nur die Datenbank-Verbindungen in den Tools.